In [2]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "code",
      "source": [
        "\"\"\"\n",
        "\n",
        "\n",
        "Network structure:\n",
        " - Inputs: x1, x2\n",
        " - Hidden layer: H1, H2 (sigmoid activation)\n",
        " - Outputs: y1, y2 (sigmoid activation)\n",
        " - Loss: Sum of squared errors (E_total = 1/2*(T1-y1)^2 + 1/2*(T2-y2)^2)\n",
        "\n",
        "\n",
        "\"\"\"\n",
        "\n",
        "import numpy as np\n",
        "\n",
        "# -------------------------------\n",
        "# 1) Setup: inputs, initial params\n",
        "# -------------------------------\n",
        "\n",
        "# Inputs (single training example)\n",
        "x1 = 0.07\n",
        "x2 = 0.06\n",
        "\n",
        "# Initial weights (input -> hidden)\n",
        "w1, w2 = 0.10, 0.20  # weights to H1 from x1, x2\n",
        "w3, w4 = 0.20, 0.30  # weights to H2 from x1, x2\n",
        "\n",
        "# Initial weights (hidden -> output)\n",
        "w5, w6 = 0.43, 0.47  # weights to y1 from H1, H2\n",
        "w7, w8 = 0.52, 0.59  # weights to y2 from H1, H2\n",
        "\n",
        "# Biases\n",
        "b1 = 0.31  # bias for hidden neurons H1, H2 (same in example)\n",
        "b2 = 0.28  # bias for output neurons y1, y2 (same in example)\n",
        "\n",
        "# Targets (desired outputs)\n",
        "T1 = 0.66\n",
        "T2 = 0.85\n",
        "\n",
        "# Learning rate\n",
        "lr = 0.5\n",
        "\n",
        "# -------------------------------\n",
        "# 2) Activation functions\n",
        "# -------------------------------\n",
        "\n",
        "def sigmoid(z):\n",
        "    \"\"\"Sigmoid activation.\"\"\"\n",
        "    return 1.0 / (1.0 + np.exp(-z))\n",
        "\n",
        "def sigmoid_derivative_from_activation(a):\n",
        "    \"\"\"Derivative of sigmoid given activation a = sigmoid(z): a*(1-a).\"\"\"\n",
        "    return a * (1.0 - a)\n",
        "\n",
        "# -------------------------------\n",
        "# 3) Helper: forward pass\n",
        "# -------------------------------\n",
        "\n",
        "def forward_pass(x1, x2, w1, w2, w3, w4, w5, w6, w7, w8, b1, b2):\n",
        "    \"\"\"\n",
        "    Compute net inputs and activations for hidden and output layers.\n",
        "    Returns a dict with nets and activations for printing/teaching.\n",
        "    \"\"\"\n",
        "    # Hidden layer linear combinations (net inputs)\n",
        "    H1_net = x1 * w1 + x2 * w2 + b1\n",
        "    H2_net = x1 * w3 + x2 * w4 + b1\n",
        "\n",
        "    # Hidden activations (sigmoid)\n",
        "    H1 = sigmoid(H1_net)\n",
        "    H2 = sigmoid(H2_net)\n",
        "\n",
        "    # Output layer linear combinations\n",
        "    y1_net = H1 * w5 + H2 * w6 + b2\n",
        "    y2_net = H1 * w7 + H2 * w8 + b2\n",
        "\n",
        "    # Output activations (sigmoid)\n",
        "    y1 = sigmoid(y1_net)\n",
        "    y2 = sigmoid(y2_net)\n",
        "\n",
        "    return {\n",
        "        \"H1_net\": H1_net, \"H2_net\": H2_net,\n",
        "        \"H1\": H1, \"H2\": H2,\n",
        "        \"y1_net\": y1_net, \"y2_net\": y2_net,\n",
        "        \"y1\": y1, \"y2\": y2\n",
        "    }\n",
        "\n",
        "# -------------------------------\n",
        "# 4) Forward before update (print step-by-step)\n",
        "# -------------------------------\n",
        "\n",
        "print(\"\\n=== FORWARD PASS (before weight update) ===\")\n",
        "out = forward_pass(x1, x2, w1, w2, w3, w4, w5, w6, w7, w8, b1, b2)\n",
        "\n",
        "print(f\"H1_net = {out['H1_net']:.7f} => H1 = sigmoid(H1_net) = {out['H1']:.9f}\")\n",
        "print(f\"H2_net = {out['H2_net']:.7f} => H2 = sigmoid(H2_net) = {out['H2']:.9f}\")\n",
        "print(f\"y1_net = {out['y1_net']:.9f} => y1 = sigmoid(y1_net) = {out['y1']:.9f}\")\n",
        "print(f\"y2_net = {out['y2_net']:.9f} => y2 = sigmoid(y2_net) = {out['y2']:.9f}\")\n",
        "\n",
        "# Compute per-output squared errors and total error\n",
        "E1 = 0.5 * (T1 - out['y1'])**2\n",
        "E2 = 0.5 * (T2 - out['y2'])**2\n",
        "E_total = E1 + E2\n",
        "print(f\"\\nE1 = 0.5*(T1 - y1)^2 = {E1:.9f}\")\n",
        "print(f\"E2 = 0.5*(T2 - y2)^2 = {E2:.9f}\")\n",
        "print(f\"Total error E_total = E1 + E2 = {E_total:.9f}\")\n",
        "\n",
        "# -------------------------------\n",
        "# 5) BACKPROPAGATION — output layer\n",
        "#    compute deltas and gradients for w5..w8\n",
        "# -------------------------------\n",
        "\n",
        "print(\"\\n=== BACKPROP: output layer ===\")\n",
        "\n",
        "# For each output neuron i: delta_i = dE/dy_i * dy_i/dnet_i\n",
        "# where dE/dy_i = -(T_i - y_i) for E = 1/2*(T-y)^2\n",
        "dE_dy1 = -(T1 - out['y1'])\n",
        "dy1_dnet = sigmoid_derivative_from_activation(out['y1'])\n",
        "delta1 = dE_dy1 * dy1_dnet  # scalar\n",
        "\n",
        "dE_dy2 = -(T2 - out['y2'])\n",
        "dy2_dnet = sigmoid_derivative_from_activation(out['y2'])\n",
        "delta2 = dE_dy2 * dy2_dnet  # scalar\n",
        "\n",
        "print(f\"dE/dy1 = {dE_dy1:.9f}, dy1/dnet = {dy1_dnet:.9f}, => delta1 = {delta1:.9f}\")\n",
        "print(f\"dE/dy2 = {dE_dy2:.9f}, dy2/dnet = {dy2_dnet:.9f}, => delta2 = {delta2:.9f}\")\n",
        "\n",
        "# Gradients for weights from hidden -> outputs:\n",
        "# dw5 = dE/dw5 = delta1 * H1\n",
        "# dw6 = dE/dw6 = delta1 * H2\n",
        "# dw7 = dE/dw7 = delta2 * H1\n",
        "# dw8 = dE/dw8 = delta2 * H2\n",
        "dw5 = delta1 * out['H1']\n",
        "dw6 = delta1 * out['H2']\n",
        "dw7 = delta2 * out['H1']\n",
        "dw8 = delta2 * out['H2']\n",
        "\n",
        "print(\"\\nGradients for hidden->output weights:\")\n",
        "print(f\"dw5 (for w5) = delta1 * H1 = {dw5:.9f}\")\n",
        "print(f\"dw6 (for w6) = delta1 * H2 = {dw6:.9f}\")\n",
        "print(f\"dw7 (for w7) = delta2 * H1 = {dw7:.9f}\")\n",
        "print(f\"dw8 (for w8) = delta2 * H2 = {dw8:.9f}\")\n",
        "\n",
        "# Update output weights (gradient descent): w <- w - lr * dw\n",
        "w5_new = w5 - lr * dw5\n",
        "w6_new = w6 - lr * dw6\n",
        "w7_new = w7 - lr * dw7\n",
        "w8_new = w8 - lr * dw8\n",
        "\n",
        "print(\"\\nUpdated hidden->output weights (one step):\")\n",
        "print(f\"w5 -> {w5_new:.9f}\")\n",
        "print(f\"w6 -> {w6_new:.9f}\")\n",
        "print(f\"w7 -> {w7_new:.9f}\")\n",
        "print(f\"w8 -> {w8_new:.9f}\")\n",
        "\n",
        "# -------------------------------\n",
        "# 6) BACKPROPAGATION — hidden layer\n",
        "#    compute deltas for H1, H2 and gradients for w1..w4\n",
        "# -------------------------------\n",
        "\n",
        "print(\"\\n=== BACKPROP: hidden layer ===\")\n",
        "\n",
        "# Error contribution from both output neurons flows back to each hidden neuron:\n",
        "# delta_H1 = (delta1*w5 + delta2*w7) * sigmoid'(H1_net)\n",
        "# delta_H2 = (delta1*w6 + delta2*w8) * sigmoid'(H2_net)\n",
        "# Note: use the original w5,w6,w7,w8 (the ones that were used in the forward pass)\n",
        "delta_H1 = (delta1 * w5 + delta2 * w7) * sigmoid_derivative_from_activation(out['H1'])\n",
        "delta_H2 = (delta1 * w6 + delta2 * w8) * sigmoid_derivative_from_activation(out['H2'])\n",
        "\n",
        "print(f\"delta_H1 = (delta1*w5 + delta2*w7) * sigmoid'(H1) = {delta_H1:.12f}\")\n",
        "print(f\"delta_H2 = (delta1*w6 + delta2*w8) * sigmoid'(H2) = {delta_H2:.12f}\")\n",
        "\n",
        "# Gradients for input->hidden weights:\n",
        "# dw1 = delta_H1 * x1, dw2 = delta_H1 * x2\n",
        "# dw3 = delta_H2 * x1, dw4 = delta_H2 * x2\n",
        "dw1 = delta_H1 * x1\n",
        "dw2 = delta_H1 * x2\n",
        "dw3 = delta_H2 * x1\n",
        "dw4 = delta_H2 * x2\n",
        "\n",
        "print(\"\\nGradients for input->hidden weights:\")\n",
        "print(f\"dw1 (for w1) = delta_H1 * x1 = {dw1:.12f}\")\n",
        "print(f\"dw2 (for w2) = delta_H1 * x2 = {dw2:.12f}\")\n",
        "print(f\"dw3 (for w3) = delta_H2 * x1 = {dw3:.12f}\")\n",
        "print(f\"dw4 (for w4) = delta_H2 * x2 = {dw4:.12f}\")\n",
        "\n",
        "# Update hidden weights\n",
        "w1_new = w1 - lr * dw1\n",
        "w2_new = w2 - lr * dw2\n",
        "w3_new = w3 - lr * dw3\n",
        "w4_new = w4 - lr * dw4\n",
        "\n",
        "print(\"\\nUpdated input->hidden weights (one step):\")\n",
        "print(f\"w1 -> {w1_new:.9f}\")\n",
        "print(f\"w2 -> {w2_new:.9f}\")\n",
        "print(f\"w3 -> {w3_new:.9f}\")\n",
        "print(f\"w4 -> {w4_new:.9f}\")\n",
        "\n",
        "# For completeness, update biases too (if you want)\n",
        "# bias at output b2 <- b2 - lr * delta (sum of deltas for outputs)\n",
        "# bias at hidden b1 <- b1 - lr * delta_H (sum of deltas for hidden neurons)\n",
        "b2_new = b2 - lr * (delta1 + delta2)  # update using both output deltas\n",
        "b1_new = b1 - lr * (delta_H1 + delta_H2)  # update using hidden deltas\n",
        "\n",
        "print(f\"\\nUpdated biases:\")\n",
        "print(f\"b1 -> {b1_new:.9f}\")\n",
        "print(f\"b2 -> {b2_new:.9f}\")\n",
        "\n",
        "# -------------------------------\n",
        "# 7) Forward pass after the update (to show error decreased)\n",
        "# -------------------------------\n",
        "\n",
        "print(\"\\n=== FORWARD PASS (after weight update) ===\")\n",
        "out_after = forward_pass(x1, x2,\n",
        "                         w1_new, w2_new, w3_new, w4_new,\n",
        "                         w5_new, w6_new, w7_new, w8_new,\n",
        "                         b1_new, b2_new)\n",
        "\n",
        "print(f\"H1 (after) = {out_after['H1']:.9f}\")\n",
        "print(f\"H2 (after) = {out_after['H2']:.9f}\")\n",
        "print(f\"y1 (after) = {out_after['y1']:.9f}\")\n",
        "print(f\"y2 (after) = {out_after['y2']:.9f}\")\n",
        "\n",
        "E1_after = 0.5 * (T1 - out_after['y1'])**2\n",
        "E2_after = 0.5 * (T2 - out_after['y2'])**2\n",
        "E_total_after = E1_after + E2_after\n",
        "\n",
        "print(f\"\\nE_total (before) = {E_total:.9f}\")\n",
        "print(f\"E_total (after)  = {E_total_after:.9f}\")\n",
        "print(\"\\n(You should see the total error decreased after one backprop step.)\")\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "zPLlQxf5EhSg",
        "outputId": "d50e79a4-0726-4fe9-bf2a-f66553dba951"
      },
      "execution_count": 14,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "=== FORWARD PASS (before weight update) ===\n",
            "H1_net = 0.3775000 => H1 = sigmoid(H1_net) = 0.593269992\n",
            "H2_net = 0.3925000 => H2 = sigmoid(H2_net) = 0.596884378\n",
            "y1_net = 1.105905967 => y1 = sigmoid(y1_net) = 0.751365070\n",
            "y2_net = 1.224921404 => y2 = sigmoid(y2_net) = 0.772928465\n",
            "\n",
            "E1 = 0.5*(T1 - y1)^2 = 0.274811083\n",
            "E2 = 0.5*(T2 - y2)^2 = 0.023560026\n",
            "Total error E_total = E1 + E2 = 0.298371109\n",
            "\n",
            "=== BACKPROP: output layer ===\n",
            "dE/dy1 = 0.741365070, dy1/dnet = 0.186815602, => delta1 = 0.138498562\n",
            "dE/dy2 = -0.217071535, dy2/dnet = 0.175510053, => delta2 = -0.038098237\n",
            "\n",
            "Gradients for hidden->output weights:\n",
            "dw5 (for w5) = delta1 * H1 = 0.082167041\n",
            "dw6 (for w6) = delta1 * H2 = 0.082667628\n",
            "dw7 (for w7) = delta2 * H1 = -0.022602540\n",
            "dw8 (for w8) = delta2 * H2 = -0.022740242\n",
            "\n",
            "Updated hidden->output weights (one step):\n",
            "w5 -> 0.358916480\n",
            "w6 -> 0.408666186\n",
            "w7 -> 0.511301270\n",
            "w8 -> 0.561370121\n",
            "\n",
            "=== BACKPROP: hidden layer ===\n",
            "delta_H1 = (delta1*w5 + delta2*w7) * sigmoid'(H1) = 0.008771354689\n",
            "delta_H2 = (delta1*w6 + delta2*w8) * sigmoid'(H2) = 0.009954254705\n",
            "\n",
            "Gradients for input->hidden weights:\n",
            "dw1 (for w1) = delta_H1 * x1 = 0.000438567734\n",
            "dw2 (for w2) = delta_H1 * x2 = 0.000877135469\n",
            "dw3 (for w3) = delta_H2 * x1 = 0.000497712735\n",
            "dw4 (for w4) = delta_H2 * x2 = 0.000995425471\n",
            "\n",
            "Updated input->hidden weights (one step):\n",
            "w1 -> 0.149780716\n",
            "w2 -> 0.199561432\n",
            "w3 -> 0.249751144\n",
            "w4 -> 0.299502287\n",
            "\n",
            "Updated biases:\n",
            "b1 -> 0.340637195\n",
            "b2 -> 0.549799837\n",
            "\n",
            "=== FORWARD PASS (after weight update) ===\n",
            "H1 (after) = 0.590995531\n",
            "H2 (after) = 0.594614536\n",
            "y1 (after) = 0.732024167\n",
            "y2 (after) = 0.765984653\n",
            "\n",
            "E_total (before) = 0.298371109\n",
            "E_total (after)  = 0.285750887\n",
            "\n",
            "(You should see the total error decreased after one backprop step.)\n"
          ]
        }
      ]
    }
  ]
}

{'nbformat': 4,
 'nbformat_minor': 0,
 'metadata': {'colab': {'provenance': []},
  'kernelspec': {'name': 'python3', 'display_name': 'Python 3'},
  'language_info': {'name': 'python'}},
 'cells': [{'cell_type': 'code',
   'source': ['"""\n',
    '\n',
    '\n',
    'Network structure:\n',
    ' - Inputs: x1, x2\n',
    ' - Hidden layer: H1, H2 (sigmoid activation)\n',
    ' - Outputs: y1, y2 (sigmoid activation)\n',
    ' - Loss: Sum of squared errors (E_total = 1/2*(T1-y1)^2 + 1/2*(T2-y2)^2)\n',
    '\n',
    '\n',
    '"""\n',
    '\n',
    'import numpy as np\n',
    '\n',
    '# -------------------------------\n',
    '# 1) Setup: inputs, initial params\n',
    '# -------------------------------\n',
    '\n',
    '# Inputs (single training example)\n',
    'x1 = 0.07\n',
    'x2 = 0.06\n',
    '\n',
    '# Initial weights (input -> hidden)\n',
    'w1, w2 = 0.10, 0.20  # weights to H1 from x1, x2\n',
    'w3, w4 = 0.20, 0.30  # weights to H2 from x1, x2\n',
    '\n',
    '# Initial